In [2]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from transformers import BertConfig, BertForSequenceClassification
import nltk
from nltk.tokenize import sent_tokenize
from keras_preprocessing.sequence import pad_sequences
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
import re

# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") # Bert-base의 토크나이저

In [3]:
data = pd.read_csv('./data_in_title.csv', dtype=object, encoding='utf-8')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6734 entries, 0 to 6733
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   content    6734 non-null   object
 1   keyword    6734 non-null   object
 2   title      6734 non-null   object
 3   url        6734 non-null   object
 4   writed_at  6734 non-null   object
 5   writer     6612 non-null   object
dtypes: object(6)
memory usage: 315.8+ KB


### bert-base 다국어

In [24]:
text = data.content.values[0]
sentences = sent_tokenize(text)
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [25]:
input_ids = pad_sequences(input_ids, maxlen=64, dtype='long', truncating='post', padding='post')

In [26]:
input_ids[0]

array([   141, 118857,    117,   8997,  15001, 119412,  37388,  14040,
         9258,  52363,  10892,  63042,  11204,    110,    117,  10413,
          110,   1792,    120,   2104,   9901,  98511,  12692,   8896,
        13890,    123,  37712,  12310,  23607,  10954,   9251,   9543,
        21386,  21275,  35506,  10739, 118766,  12605,   9638,  38631,
         9405,  26784,  13890,    119,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

In [27]:
from transformers import BertConfig, BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [29]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
batch_size = 32

# 학습 데이터셋을 TensorDataset으로 변환
train_dataset = TensorDataset(torch.tensor(input_ids_train),
                              torch.tensor(attention_masks_train),
                              torch.tensor(labels_train))

# 학습 데이터셋을 랜덤 샘플러로 사용하여 데이터 로더 생성
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset,
                              sampler=train_sampler,
                              batch_size=batch_size)

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = len(train_dataloader) * 10)

for i in range(10):
    total_loss = 0
    # 모델을 학습모드로 변경
    model.train()
    for step, batch in enumerate(train_dataloader):
        # 500 스텝마다 시간 출력
        # if step % 500 == 0 and not step == 0:
        #     elapsed = format_time(time.time()- t0)
        
        # batch의 데이터를 device에 올림
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)
        loss = outputs[0]
        total_loss += loss
        loss.backward()# loss 기울기 계산
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)# 기울기 폭발 방지
        optimizer.step()# 파라미터 업데이트
        scheduler.step()# 학습률 업데이트
        model.zero_grad()# 기울기 초기화
    avg_train_loss = total_loss / len(train_dataloader)

### kr-finbert

In [32]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")

model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")

In [33]:
text = data.content.values[0]
sentences = sent_tokenize(text)
sentences

['D램, 낸드플래시 매출은 각각 37%, 31% ↓ / 中 파운드리 공장 2분기 2020년 말 양산 \nSK하이닉스 이천 사업장.',
 '연합뉴스 SK하이닉스의 지난해 매출액이 전년 대비 33% 감소했지만, 비메모리 반도체 매출은 50%가량 증가한 것으로 나타났다.',
 '31일 SK하이닉스의 사업보고서에 따르면, 2019년 D램 매출은 20조3000억원으로 전년과 비교해 37% 감소했고, 낸드플래시 매출은 5조1000억원으로 31% 줄어든 것으로 집계됐다.',
 '반면 비메모리 사업을 포함한 기타 항목 매출은 2018년 6500억원에서 지난해 1조6000억원으로 2배 이상 증가했다.',
 '지난해 기타 매출 가운데는 SK하이닉스가 2018년 12월 지분 100%를 인수한 사회적기업 행복나래 등 자회사 매출 약 7000억원이 포함됐다.',
 '나머지 매출 약 8000억원은 이미지센서 부문과 파운드리 자회사 SK하이닉스시스템IC 등 비메모리 반도체 사업으로, 2018년(5500억여원) 대비 성장한 것으로 파악된다.',
 '이에 따라 D램 부문 매출 비중은 2018년 80%에서 지난해 75%로 줄어들었고, 기타 항목 비중은 같은 기간 2%에서 6%로 확대됐다.',
 'SK하이닉스가 주력하는 분야는 비메모리 사업 중에서도 모바일, 노트북용 이미지센서 시장이다.',
 '지난해 일본에 차세대 CIS(CMOS 이미지센서) 연구개발센터를 여는 등 시장 점유율 확대를 위한 제품 개발에 주력하고 있다.',
 "올해는 모든 CIS 제품을 '블랙펄(Black Pearl)'로 공식 브랜딩하고, 하반기 중 픽셀 크기로 4800만화소를 구현한 제품도 선보인다는 계획이다.",
 'SK하이닉스시스템IC는 중국 장쑤성 우시에서 합작법인을 설립해 파운드리 공장을 건설하고 있다.',
 '올해 2분기 내 준공되면 연말 양산이 목표다.',
 'SK하이닉스는 이날 매그나칩반도체의 파운드리(반도체 위탁생산) 부문도 인수했다.',
 '박세준 기자 세상을 보는 눈, 세계일보']

In [34]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=128, dtype='long', truncating='post', padding='post')

In [35]:
tokenized_texts

[['D',
  '##램',
  ',',
  '낸',
  '##드',
  '##플',
  '##래',
  '##시',
  '매출',
  '##은',
  '각각',
  '37',
  '%',
  ',',
  '31',
  '%',
  '↓',
  '/',
  '中',
  '파',
  '##운드',
  '##리',
  '공장',
  '2',
  '##분기',
  '2020년',
  '말',
  '양산',
  'SK',
  '##하이',
  '##닉',
  '##스',
  '이',
  '##천',
  '사업',
  '##장',
  '.'],
 ['연합',
  '##뉴스',
  'SK',
  '##하이',
  '##닉',
  '##스의',
  '지난해',
  '매출',
  '##액이',
  '전년',
  '대비',
  '33',
  '%',
  '감소',
  '##했지만',
  ',',
  '비',
  '##메',
  '##모',
  '##리',
  '반도체',
  '매출',
  '##은',
  '50',
  '%',
  '가',
  '##량',
  '증가',
  '##한',
  '것으로',
  '나타났다',
  '.'],
 ['31일',
  'SK',
  '##하이',
  '##닉',
  '##스의',
  '사업',
  '##보고',
  '##서에',
  '따르면',
  ',',
  '2019년',
  'D',
  '##램',
  '매출',
  '##은',
  '20',
  '##조',
  '##30',
  '##00',
  '##억원',
  '##으로',
  '전년',
  '##과',
  '비교',
  '##해',
  '37',
  '%',
  '감소',
  '##했고',
  ',',
  '낸',
  '##드',
  '##플',
  '##래',
  '##시',
  '매출',
  '##은',
  '5',
  '##조',
  '##100',
  '##0',
  '##억원',
  '##으로',
  '31',
  '%',
  '줄어',
  '##든',
  '것으로',
 

In [36]:
max_len = 0
for sen_ids in input_ids:
    now_len = len(sen_ids)
    if now_len > max_len:
        max_len = now_len
max_len

128

In [ ]:
input_ids

In [13]:
# BERT의 단어 집합을 vocabulary.txt에 저장
with open('./finbert_vocabulary.txt', 'w') as f:
    for token in tokenizer.vocab.keys():
        f.write(token + '\n')

In [17]:
from transformers import pipeline
classifier  = pipeline('sentiment-analysis', model='snunlp/KR-FinBert-SC')
result = classifier(text)
print(result)

[{'label': 'positive', 'score': 0.818943202495575}]


In [18]:
text

"D램, 낸드플래시 매출은 각각 37%, 31% ↓ / 中 파운드리 공장 2분기 2020년 말 양산 \nSK하이닉스 이천 사업장. 연합뉴스 SK하이닉스의 지난해 매출액이 전년 대비 33% 감소했지만, 비메모리 반도체 매출은 50%가량 증가한 것으로 나타났다. 31일 SK하이닉스의 사업보고서에 따르면, 2019년 D램 매출은 20조3000억원으로 전년과 비교해 37% 감소했고, 낸드플래시 매출은 5조1000억원으로 31% 줄어든 것으로 집계됐다. 반면 비메모리 사업을 포함한 기타 항목 매출은 2018년 6500억원에서 지난해 1조6000억원으로 2배 이상 증가했다. 지난해 기타 매출 가운데는 SK하이닉스가 2018년 12월 지분 100%를 인수한 사회적기업 행복나래 등 자회사 매출 약 7000억원이 포함됐다. 나머지 매출 약 8000억원은 이미지센서 부문과 파운드리 자회사 SK하이닉스시스템IC 등 비메모리 반도체 사업으로, 2018년(5500억여원) 대비 성장한 것으로 파악된다. 이에 따라 D램 부문 매출 비중은 2018년 80%에서 지난해 75%로 줄어들었고, 기타 항목 비중은 같은 기간 2%에서 6%로 확대됐다. SK하이닉스가 주력하는 분야는 비메모리 사업 중에서도 모바일, 노트북용 이미지센서 시장이다. 지난해 일본에 차세대 CIS(CMOS 이미지센서) 연구개발센터를 여는 등 시장 점유율 확대를 위한 제품 개발에 주력하고 있다. 올해는 모든 CIS 제품을 '블랙펄(Black Pearl)'로 공식 브랜딩하고, 하반기 중 픽셀 크기로 4800만화소를 구현한 제품도 선보인다는 계획이다. SK하이닉스시스템IC는 중국 장쑤성 우시에서 합작법인을 설립해 파운드리 공장을 건설하고 있다. 올해 2분기 내 준공되면 연말 양산이 목표다. SK하이닉스는 이날 매그나칩반도체의 파운드리(반도체 위탁생산) 부문도 인수했다. 박세준 기자 세상을 보는 눈, 세계일보"

In [19]:
help(pipeline)

Help on function pipeline in module transformers.pipelines:

pipeline(task: str = None, model: Optional = None, config: Union[str, transformers.configuration_utils.PretrainedConfig, NoneType] = None, tokenizer: Union[str, transformers.tokenization_utils.PreTrainedTokenizer, transformers.tokenization_utils_fast.PreTrainedTokenizerFast, NoneType] = None, feature_extractor: Union[str, ForwardRef('SequenceFeatureExtractor'), NoneType] = None, framework: Optional[str] = None, revision: Optional[str] = None, use_fast: bool = True, use_auth_token: Union[bool, str, NoneType] = None, device: Union[int, str, ForwardRef('torch.device'), NoneType] = None, device_map=None, torch_dtype=None, trust_remote_code: Optional[bool] = None, model_kwargs: Dict[str, Any] = None, pipeline_class: Optional[Any] = None, **kwargs) -> transformers.pipelines.base.Pipeline
    Utility factory method to build a [`Pipeline`].
    
    Pipelines are made of:
    
        - A [tokenizer](tokenizer) in charge of mapping r

In [55]:
# 특수문자 하나하나가 토큰이라서 특수문자 제거하고 해야함

encoded_dict = tokenizer.encode_plus(
    text=text, 
    add_special_tokens=True, # cls, sep 등
    max_length=64, # 최대 길이 설정
    pad_to_max_length=True, # 패딩 여부
    return_attention_mask=True, # 패딩한 부분은 0으로 처리하는 마스크
    truncation=True
)# 기본적으로 뒤를 자르게 설정됨

c:\Users\tlrks\anaconda3\envs\study\lib\site-packages\transformers\tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [56]:
encoded_dict.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
encoded_dict['input_ids']

In [ ]:
encoded_dict['attention_mask']

In [ ]:
encoded_dict['token_type_ids']

### 0426

In [3]:
data = pd.read_csv('./data_in_title.csv', dtype=object)
data.head()

,content,keyword,title,url,writed_at,writer
0,"D램, 낸드플래시 매출은 각각 37%, 31% ↓ / 中 파운드리 공장 2분기 20...",하이닉스,"SK하이닉스, 2019년 비메모리 매출 50% ↑",https://n.news.naver.com/mnews/article/022/000...,2020-04-01 03:04:05,박세준
1,"[이데일리 최정희 기자] 삼성전자(005930), SK하이닉스(000660) 주가가...",하이닉스,[특징주]1분기 실적 호조에 삼성전자·SK하이닉스 강세,https://n.news.naver.com/mnews/article/018/000...,2020-04-07 09:16:04,최정희
2,SK하이닉스[연합뉴스TV 제공] (서울=연합뉴스) 김기훈 기자 = 유진투자증권은 1...,하이닉스,"유진투자 ""하반기 메모리 가격 하락 전망…SK하이닉스 목표가↓""",https://n.news.naver.com/mnews/article/001/001...,2020-04-14 08:39:00,김기훈
3,[머니투데이 김태현 기자] 삼성증권은 SK하이닉스에 대해 환율효과와 낸드마진 등으로...,하이닉스,"SK하이닉스, 낸드마진 개선…1Q 호실적 기대-삼성",https://n.news.naver.com/mnews/article/008/000...,2020-04-20 08:35:37,김태현
4,SK하이닉스가 메모리 반도체 가격 상승에 힘입어 올 1분기 영업이익 8000억원을 ...,하이닉스,SK하이닉스 코로나에도 '어닝 서프라이즈'...1분기 영업이익 8003억원,https://n.news.naver.com/mnews/article/366/000...,2020-04-23 08:37:21,윤민혁


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6734 entries, 0 to 6733
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   content    6734 non-null   object
 1   keyword    6734 non-null   object
 2   title      6734 non-null   object
 3   url        6734 non-null   object
 4   writed_at  6734 non-null   object
 5   writer     6612 non-null   object
dtypes: object(6)
memory usage: 315.8+ KB


In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

#https://github.com/huggingface/transformers/issues/5421

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [26]:
tokenizer = BertTokenizer.from_pretrained("snunlp/KR-FinBert-SC")

model = BertForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")

#### 368page~ 텐서플로 예시

In [17]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

def bert_tokenizer(sent, MAX_LEN, pt=None):
    encoded_dict = tokenizer.encode_plus(
        text=sent,
        add_special_tokens = True,
        max_length = MAX_LEN,
        padding = True,
        return_attention_mask = True,
        truncation = True,
        return_tensors = pt
    )

    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']

    return input_id, attention_mask, token_type_id

In [12]:
text = data.content.values[0]

In [13]:
bert_tokenizer(text, 64, 'pt')

(tensor([[   101,    141, 118857,    117,   8997,  15001, 119412,  37388,  14040,
            9258,  52363,  10892,  63042,  11204,    110,    117,  10413,    110,
            1792,    120,   2104,   9901,  98511,  12692,   8896,  13890,    123,
           37712,  12310,  23607,  10954,   9251,   9543,  21386,  21275,  35506,
           10739, 118766,  12605,   9638,  38631,   9405,  26784,  13890,    119,
           94224,  21275,  35506,  10739, 118766,  49319,   9706,  33305,  14523,
            9258,  52363, 119122,  10739,   9665,  10954,   9069,  29455,  11000,
             102]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 

In [21]:
print(bert_tokenizer(text, 64))

([101, 141, 118857, 117, 8997, 15001, 119412, 37388, 14040, 9258, 52363, 10892, 63042, 11204, 110, 117, 10413, 110, 1792, 120, 2104, 9901, 98511, 12692, 8896, 13890, 123, 37712, 12310, 23607, 10954, 9251, 9543, 21386, 21275, 35506, 10739, 118766, 12605, 9638, 38631, 9405, 26784, 13890, 119, 94224, 21275, 35506, 10739, 118766, 49319, 9706, 33305, 14523, 9258, 52363, 119122, 10739, 9665, 10954, 9069, 29455, 11000, 102], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [22]:
print(tokenizer.all_special_tokens,"\n",tokenizer.all_special_ids)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'] 
 [100, 102, 0, 101, 103]


In [28]:
train = data.content.values[:3]
train_labels = np.array([1,1,0])

In [29]:
input_ids = []
attention_masks = []
token_type_ids = []
train_data_labels = []

def clean_text(sent):
    sent_clean = re.sub("[^가-힣a-zA-Z0-9 ]", " ", sent)
    return sent_clean

for train_sent, train_label in zip(train, train_labels):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(clean_text(train_sent), 128)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        train_data_labels.append(train_label)
    except Exception as e:
        print(e)
        print(train_sent)

input_ids = np.array(input_ids, dtype=int)
attention_masks = np.array(attention_masks, dtype=int)
token_type_ids = np.array(token_type_ids, dtype=int)
inputs = (input_ids, attention_masks, token_type_ids)

labels = np.array(train_data_labels, dtype=np.int8)

In [32]:
tokenizer.decode(input_ids[0])

'[CLS] D램 낸드플래시 매출은 각각 37 31 파운드리 공장 2분기 2020년 말 양산 SK하이닉스 이천 사업장 연합뉴스 SK하이닉스의 지난해 매출액이 전년 대비 33 감소했지만 비메모리 반도체 매출은 50 가량 증가한 것으로 나타났다 31일 SK하이닉스의 사업보고서에 따르면 2019년 D램 매출은 20조3000억원으로 전년과 비교해 37 감소했고 낸드플래시 매출은 5조1000억 [SEP]'

In [12]:
import os

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
# import tensorflow_models as tfm
# import tensorflow_hub as hub
# import tensorflow_datasets as tfds
# tfds.disable_progress_bar()

#### tf install

In [11]:
!pip3 install tf-models-official

  Using cached tf_models_official-2.12.0-py2.py3-none-any.whl (2.6 MB)
  Using cached immutabledict-2.2.4-py3-none-any.whl (4.1 kB)
  Using cached tf_models_official-2.11.5-py2.py3-none-any.whl (2.4 MB)
  Using cached tensorflow-2.11.1-cp310-cp310-win_amd64.whl (1.9 kB)
  Using cached tf_models_official-2.11.4-py2.py3-none-any.whl (2.4 MB)
  Using cached tf_models_official-2.11.3-py2.py3-none-any.whl (2.3 MB)
  Using cached tf_models_official-2.11.2-py2.py3-none-any.whl (2.3 MB)
  Using cached tf_models_official-2.11.0-py2.py3-none-any.whl (2.3 MB)
  Using cached tf_models_official-2.10.1-py2.py3-none-any.whl (2.2 MB)
  Using cached tensorflow_text-2.10.0-cp310-cp310-win_amd64.whl (5.0 MB)
  Using cached tensorflow-2.10.1-cp310-cp310-win_amd64.whl (455.9 MB)
  Using cached sacrebleu-2.2.0-py3-none-any.whl (116 kB)
  Using cached protobuf-3.19.6-cp310-cp310-win_amd64.whl (895 kB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorboard-2.10.1-py3-none-any.whl

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\tlrks\\anaconda3\\envs\\study\\lib\\site-packages\\tensorflow_estimator-2.10.0.dist-info\\METADATA'


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install tensorflow-datasets


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install --upgrade tensorflow_hub

  Using cached tensorflow_hub-0.13.0-py2.py3-none-any.whl (100 kB)



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
!pip3 install tensorflow-text

  Using cached tensorflow_text-2.10.0-cp310-cp310-win_amd64.whl (5.0 MB)
     ---------------------------------------- 0.0/455.9 MB ? eta -:--:--
     --------------------------------------- 0.4/455.9 MB 11.6 MB/s eta 0:00:40
     --------------------------------------- 0.9/455.9 MB 11.6 MB/s eta 0:00:40
     --------------------------------------- 1.4/455.9 MB 11.4 MB/s eta 0:00:40
     --------------------------------------- 1.8/455.9 MB 10.6 MB/s eta 0:00:43
     --------------------------------------- 2.4/455.9 MB 10.8 MB/s eta 0:00:42
     --------------------------------------- 2.9/455.9 MB 10.9 MB/s eta 0:00:42
     --------------------------------------- 3.4/455.9 MB 11.5 MB/s eta 0:00:40
     --------------------------------------- 4.0/455.9 MB 11.5 MB/s eta 0:00:40
     --------------------------------------- 4.5/455.9 MB 11.4 MB/s eta 0:00:40
     --------------------------------------- 5.0/455.9 MB 11.5 MB/s eta 0:00:40
     --------------------------------------- 5.6/455.9

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\tlrks\\anaconda3\\envs\\study\\lib\\site-packages\\tensorflow_estimator-2.10.0.dist-info\\METADATA'


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 0427

In [14]:
import tensorflow as tf
from transformers import TFBertModel
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.10.0


In [15]:
class TFBertClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(
            num_class, 
            kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range),
            name="classifier",
        )

    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        # sequence output, pooled output, (hidden, attentions)로 출력
        outputs = self.bert(
            inputs, attention_mask=attention_mask, token_type_ids=token_type_ids
        )
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

In [16]:
cls_model = TFBertClassifier(model_name="bert-base-multilingual-cased",
                             dir_path='bert-ckpt',
                             num_class=2)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

c:\Users\tlrks\anaconda3\envs\study\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tlrks\Desktop\workspace\exercise\BERT\bert-ckpt. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [20]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

cls_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

20년 4월 2일 종가 / 4월 3일 종가 / 4월 4일 종가
기사데이터가 4월 3일 12시 데이터라면 -> 4월 3일 당일 종가를 가져옴 //
15시40분 이전 데이터 -> 4월 2일종가랑 비교 -> 오르면 1, 떨어졌으면 0
15시40분 이후 데이터 -> 4월 4일 종가랑 비교 -> 4일에 올랐으면 1, 떨어졌으면 0

주가데이터프레임에서 row를 하나씩 가져옴 -> 다음 row랑 종가를 비교 -> 0인지, 1인지 결정
4월 3일 종가를 가져왔고 다음날이 4월 6일로 되어 있으면, 4월 3일 15:40 이후, 4월 6일 15:40 이전의 기사들 데이터에 대해 위에서 결정한 라벨을 부여
그다음은 4월 6일..

In [21]:
text

NameError: name 'text' is not defined

In [29]:
text = data.content.values[0]
text

"D램, 낸드플래시 매출은 각각 37%, 31% ↓ / 中 파운드리 공장 2분기 2020년 말 양산 \nSK하이닉스 이천 사업장. 연합뉴스 SK하이닉스의 지난해 매출액이 전년 대비 33% 감소했지만, 비메모리 반도체 매출은 50%가량 증가한 것으로 나타났다. 31일 SK하이닉스의 사업보고서에 따르면, 2019년 D램 매출은 20조3000억원으로 전년과 비교해 37% 감소했고, 낸드플래시 매출은 5조1000억원으로 31% 줄어든 것으로 집계됐다. 반면 비메모리 사업을 포함한 기타 항목 매출은 2018년 6500억원에서 지난해 1조6000억원으로 2배 이상 증가했다. 지난해 기타 매출 가운데는 SK하이닉스가 2018년 12월 지분 100%를 인수한 사회적기업 행복나래 등 자회사 매출 약 7000억원이 포함됐다. 나머지 매출 약 8000억원은 이미지센서 부문과 파운드리 자회사 SK하이닉스시스템IC 등 비메모리 반도체 사업으로, 2018년(5500억여원) 대비 성장한 것으로 파악된다. 이에 따라 D램 부문 매출 비중은 2018년 80%에서 지난해 75%로 줄어들었고, 기타 항목 비중은 같은 기간 2%에서 6%로 확대됐다. SK하이닉스가 주력하는 분야는 비메모리 사업 중에서도 모바일, 노트북용 이미지센서 시장이다. 지난해 일본에 차세대 CIS(CMOS 이미지센서) 연구개발센터를 여는 등 시장 점유율 확대를 위한 제품 개발에 주력하고 있다. 올해는 모든 CIS 제품을 '블랙펄(Black Pearl)'로 공식 브랜딩하고, 하반기 중 픽셀 크기로 4800만화소를 구현한 제품도 선보인다는 계획이다. SK하이닉스시스템IC는 중국 장쑤성 우시에서 합작법인을 설립해 파운드리 공장을 건설하고 있다. 올해 2분기 내 준공되면 연말 양산이 목표다. SK하이닉스는 이날 매그나칩반도체의 파운드리(반도체 위탁생산) 부문도 인수했다. 박세준 기자 세상을 보는 눈, 세계일보"

In [24]:
ttt = re.sub('↓', '하락', text)
ttt

# ttt = re.sub('[^가-힣a-zA-z0-9 ]', ' ', text)
# ttt

'D램  낸드플래시 매출은 각각 37   31        파운드리 공장 2분기 2020년 말 양산  SK하이닉스 이천 사업장  연합뉴스 SK하이닉스의 지난해 매출액이 전년 대비 33  감소했지만  비메모리 반도체 매출은 50 가량 증가한 것으로 나타났다  31일 SK하이닉스의 사업보고서에 따르면  2019년 D램 매출은 20조3000억원으로 전년과 비교해 37  감소했고  낸드플래시 매출은 5조1000억원으로 31  줄어든 것으로 집계됐다  반면 비메모리 사업을 포함한 기타 항목 매출은 2018년 6500억원에서 지난해 1조6000억원으로 2배 이상 증가했다  지난해 기타 매출 가운데는 SK하이닉스가 2018년 12월 지분 100 를 인수한 사회적기업 행복나래 등 자회사 매출 약 7000억원이 포함됐다  나머지 매출 약 8000억원은 이미지센서 부문과 파운드리 자회사 SK하이닉스시스템IC 등 비메모리 반도체 사업으로  2018년 5500억여원  대비 성장한 것으로 파악된다  이에 따라 D램 부문 매출 비중은 2018년 80 에서 지난해 75 로 줄어들었고  기타 항목 비중은 같은 기간 2 에서 6 로 확대됐다  SK하이닉스가 주력하는 분야는 비메모리 사업 중에서도 모바일  노트북용 이미지센서 시장이다  지난해 일본에 차세대 CIS CMOS 이미지센서  연구개발센터를 여는 등 시장 점유율 확대를 위한 제품 개발에 주력하고 있다  올해는 모든 CIS 제품을  블랙펄 Black Pearl  로 공식 브랜딩하고  하반기 중 픽셀 크기로 4800만화소를 구현한 제품도 선보인다는 계획이다  SK하이닉스시스템IC는 중국 장쑤성 우시에서 합작법인을 설립해 파운드리 공장을 건설하고 있다  올해 2분기 내 준공되면 연말 양산이 목표다  SK하이닉스는 이날 매그나칩반도체의 파운드리 반도체 위탁생산  부문도 인수했다  박세준 기자 세상을 보는 눈  세계일보'

In [30]:
tokenizer.tokenize(text)

['D',
 '##램',
 ',',
 '낸',
 '##드',
 '##플',
 '##래',
 '##시',
 '매출',
 '##은',
 '각각',
 '37',
 '%',
 ',',
 '31',
 '%',
 '↓',
 '/',
 '中',
 '파',
 '##운드',
 '##리',
 '공장',
 '2',
 '##분기',
 '2020년',
 '말',
 '양산',
 'SK',
 '##하이',
 '##닉',
 '##스',
 '이',
 '##천',
 '사업',
 '##장',
 '.',
 '연합',
 '##뉴스',
 'SK',
 '##하이',
 '##닉',
 '##스의',
 '지난해',
 '매출',
 '##액이',
 '전년',
 '대비',
 '33',
 '%',
 '감소',
 '##했지만',
 ',',
 '비',
 '##메',
 '##모',
 '##리',
 '반도체',
 '매출',
 '##은',
 '50',
 '%',
 '가',
 '##량',
 '증가',
 '##한',
 '것으로',
 '나타났다',
 '.',
 '31일',
 'SK',
 '##하이',
 '##닉',
 '##스의',
 '사업',
 '##보고',
 '##서에',
 '따르면',
 ',',
 '2019년',
 'D',
 '##램',
 '매출',
 '##은',
 '20',
 '##조',
 '##30',
 '##00',
 '##억원',
 '##으로',
 '전년',
 '##과',
 '비교',
 '##해',
 '37',
 '%',
 '감소',
 '##했고',
 ',',
 '낸',
 '##드',
 '##플',
 '##래',
 '##시',
 '매출',
 '##은',
 '5',
 '##조',
 '##100',
 '##0',
 '##억원',
 '##으로',
 '31',
 '%',
 '줄어',
 '##든',
 '것으로',
 '집계',
 '##됐다',
 '.',
 '반면',
 '비',
 '##메',
 '##모',
 '##리',
 '사업을',
 '포함한',
 '기타',
 '항',
 '##목',
 '매출',
 '##은',
 '2018년',

60개 문서가 있으면 0, 1 결과 60개
얘를 평균